## Quick import and load files

In [1]:
# set path to the parent dir
import os
import sys
sys.path.append('src')
os.chdir(os.path.join(os.path.expanduser('~'),'project/qstar/SWE-agent'))

!pwd


/home/jas/detr_blob/jas/project/qstar/SWE-agent


In [2]:
from IPython.display import Markdown

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2


# 1.0 - Load the agent and the enviroment 

> similar to the run.py file in the main folder

In [3]:
import json
import logging
import os
import re
import traceback
import yaml

from dataclasses import dataclass
from getpass import getuser
from pathlib import Path
from rich.logging import RichHandler
from simple_parsing import parse
from simple_parsing.helpers import Serializable, FrozenSerializable, FlattenedAccess
from sweagent import (
    Agent,
    AgentArguments,
    EnvironmentArguments,
    ModelArguments,
    SWEEnv,
    get_data_path_name,
)
from swebench import KEY_INSTANCE_ID, KEY_MODEL, KEY_PREDICTION
from unidiff import PatchSet

handler = RichHandler(show_time=False, show_path=False)
handler.setLevel(logging.DEBUG)
logger = logging.getLogger("run_dev")
logger.setLevel(logging.DEBUG)
logger.addHandler(handler)
logger.propagate = False
logging.getLogger("simple_parsing").setLevel(logging.WARNING)


@dataclass(frozen=False)
class ScriptArguments(FlattenedAccess, Serializable):#FrozenSerializable):
    environment: EnvironmentArguments
    agent: AgentArguments
    instance_filter: str = ".*"  # Only run instances that completely match this regex
    skip_existing: bool = True  # Skip instances with existing trajectories
    suffix: str = ""

    @property
    def run_name(self):
        """Generate a unique name for this run based on the arguments."""
        model_name = args.agent.model.model_name.replace(":", "-")
        data_stem = get_data_path_name(args.environment.data_path)
        config_stem = Path(args.agent.config_file).stem

        temp = args.agent.model.temperature
        top_p = args.agent.model.top_p

        per_instance_cost_limit = args.agent.model.per_instance_cost_limit
        install_env = args.environment.install_environment

        return (
            f"{model_name}__{data_stem}__{config_stem}__t-{temp:.2f}__p-{top_p:.2f}"
            + f"__c-{per_instance_cost_limit:.2f}__install-{int(install_env)}"
            + (f"__{self.suffix}" if self.suffix else "")
        )


def main(args: ScriptArguments):
    logger.info(f"📙 Arguments: {args.dumps_yaml()}")
    agent = Agent("primary", args.agent)

    env = SWEEnv(args.environment)

    traj_dir = Path("trajectories") / Path(getuser()) / args.run_name
    os.makedirs(traj_dir, exist_ok=True)

    save_arguments(traj_dir, args)

    for index in range(len(env.data)):
        try:
            # Reset environment
            instance_id = env.data[index]["instance_id"]
            if should_skip(args, traj_dir, instance_id):
                continue
            logger.info("▶️  Beginning task " + str(index))

            observation, info = env.reset(index)
            if info is None:
                continue

            # Get info, patch information
            issue = getattr(env, "query", None)
            files = []
            if "patch" in env.record:
                files = "\n".join(
                    [f"- {x.path}" for x in PatchSet(env.record["patch"]).modified_files]
                )
            # Get test files, F2P tests information
            test_files = []
            if "test_patch" in env.record:
                test_patch_obj = PatchSet(env.record["test_patch"])
                test_files = "\n".join(
                    [f"- {x.path}" for x in test_patch_obj.modified_files + test_patch_obj.added_files]
                )
            tests = ""
            if "FAIL_TO_PASS" in env.record:
                tests = "\n".join([f"- {x}" for x in env.record["FAIL_TO_PASS"]])

            setup_args = {
                "issue": issue,
                "files": files,
                "test_files": test_files,
                "tests": tests
            }
            info = agent.run(
                setup_args=setup_args,
                env=env,
                observation=observation,
                traj_dir=traj_dir,
                return_type="info",
            )
            save_predictions(traj_dir, instance_id, info)

        except KeyboardInterrupt:
            logger.info("Exiting InterCode environment...")
            env.close()
            break
        except Exception as e:
            traceback.print_exc()
            logger.warning(f"❌ Failed on {env.record['instance_id']}: {e}")
            env.reset_container()
            continue


def save_arguments(traj_dir, args):
    """Save the arguments to a yaml file to the run's trajectory directory."""
    log_path = traj_dir / "args.yaml"

    if log_path.exists():
        try:
            other_args = args.load_yaml(log_path)
            if (args.dumps_yaml() != other_args.dumps_yaml()):  # check yaml equality instead of object equality
                logger.warning("**************************************************")
                logger.warning("Found existing args.yaml with different arguments!")
                logger.warning("**************************************************")
        except Exception as e:
            logger.warning(f"Failed to load existing args.yaml: {e}")

    with log_path.open("w") as f:
        args.dump_yaml(f)


def should_skip(args, traj_dir, instance_id):
    """Check if we should skip this instance based on the instance filter and skip_existing flag."""
    # Skip instances that don't match the instance filter
    if re.match(args.instance_filter, instance_id) is None:
        logger.info(f"Instance filter not matched. Skipping instance {instance_id}")
        return True

    # If flag is set to False, don't skip
    if not args.skip_existing:
        return False

    # Check if there's an existing trajectory for this instance
    log_path = traj_dir / (instance_id + ".traj")
    if log_path.exists():
        with log_path.open("r") as f:
            data = json.load(f)
        # If the trajectory has no exit status, it's incomplete and we will redo it
        exit_status = data["info"].get("exit_status", None)
        if exit_status == "early_exit" or exit_status is None:
            logger.info(f"Found existing trajectory with no exit status: {log_path}")
            logger.info("Removing incomplete trajectory...")
            os.remove(log_path)
        else:
            logger.info(f"⏭️ Skipping existing trajectory: {log_path}")
            return True
    return False


def save_predictions(traj_dir, instance_id, info):
    output_file = Path(traj_dir) / "all_preds.jsonl"
    model_patch = info["submission"] if "submission" in info else None
    datum = {
        KEY_MODEL: Path(traj_dir).name,
        KEY_INSTANCE_ID: instance_id,
        KEY_PREDICTION: model_patch,
    }
    with open(output_file, "a+") as fp:
        print(json.dumps(datum), file=fp, flush=True)
    logger.info(f"Saved predictions to {output_file}")    


/home/jas/anaconda3/envs/swe-agent/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
defaults = ScriptArguments(
        suffix="",
        environment=EnvironmentArguments(
            image_name="swe-agent",
            data_path="princeton-nlp/SWE-bench_Lite",
            split="dev",
            verbose=True,
            install_environment=True,
        ),
        skip_existing=True,
        agent=AgentArguments(
            model=ModelArguments(
                model_name="gpt4",
                total_cost_limit=0.0,
                per_instance_cost_limit=2.0,
                temperature=0.2,
                top_p=0.95,
            ),
            config_file="config/default.yaml",
        ),
    )


In [5]:
# Nicer yaml dumping of multiline strings
def multiline_representer(dumper, data):
    """configures yaml for dumping multiline strings
    Ref: https://stackoverflow.com/questions/8640959/how-can-i-control-what-scalar-form-pyyaml-uses-for-my-data
    """
    if data.count("\n") > 0:  # check for multiline string
        return dumper.represent_scalar("tag:yaml.org,2002:str", data, style="|")
    return dumper.represent_scalar("tag:yaml.org,2002:str", data)

yaml.add_representer(str, multiline_representer)


In [6]:
from simple_parsing import parse_known_args


In [7]:
args = parse_known_args(ScriptArguments, default=defaults, add_config_path_arg=False)
a = args[0] 

# for instance level testing
# a.data_path = "https://github.com/pvlib/pvlib-python/issues/1603"
# a.config_file = "config/default_from_url.yaml"

# for swe-bench-lite testing
a.config_file = "config/default.yaml"

args = a
args


ScriptArguments(environment=EnvironmentArguments(data_path='princeton-nlp/SWE-bench_Lite', image_name='swe-agent', split='dev', base_commit=None, container_name=None, install_environment=True, timeout=35, verbose=True, no_mirror=False, environment_setup=None), agent=AgentArguments(model=ModelArguments(model_name='gpt4', per_instance_cost_limit=2.0, total_cost_limit=0.0, temperature=0.2, top_p=0.95, replay_path=None, host_url='localhost:11434'), config_file='config/default.yaml', config=AgentConfig(system_template="SETTING: You are an autonomous programmer, and you're working directly in the command line with a special interface.\n\nThe special interface consists of a file editor that shows you {WINDOW} lines of a file at a time.\nIn addition to typical bash commands, you can also use the following commands to help you navigate and edit files.\n\nCOMMANDS:\n{command_docs}\n\nPlease note that THE EDIT COMMAND REQUIRES PROPER INDENTATION. \nIf you'd like to add the line '        print(x)'

In [22]:
# main(args)


In [8]:
def test_env(args: ScriptArguments):
    logger.info(f"📙 Arguments: {args.dumps_yaml()}")
    agent = Agent("primary", args.agent)

    env = SWEEnv(args.environment)

    traj_dir = Path("trajectories") / Path(getuser()) / args.run_name
    os.makedirs(traj_dir, exist_ok=True)

    save_arguments(traj_dir, args)

    for index in range(len(env.data)):
        try:
            # Reset environment
            instance_id = env.data[index]["instance_id"]
            if should_skip(args, traj_dir, instance_id):
                continue
            logger.info("▶️  Beginning task " + str(index))

            observation, info = env.reset(index)
            if info is None:
                continue

            # Get info, patch information
            issue = getattr(env, "query", None)
            files = []
            if "patch" in env.record:
                files = "\n".join(
                    [f"- {x.path}" for x in PatchSet(env.record["patch"]).modified_files]
                )
            # Get test files, F2P tests information
            test_files = []
            if "test_patch" in env.record:
                test_patch_obj = PatchSet(env.record["test_patch"])
                test_files = "\n".join(
                    [f"- {x.path}" for x in test_patch_obj.modified_files + test_patch_obj.added_files]
                )
            tests = ""
            if "FAIL_TO_PASS" in env.record:
                tests = "\n".join([f"- {x}" for x in env.record["FAIL_TO_PASS"]])

            setup_args = {
                "issue": issue,
                "files": files,
                "test_files": test_files,
                "tests": tests
            }
            info = agent.run(
                setup_args=setup_args,
                env=env,
                observation=observation,
                traj_dir=traj_dir,
                return_type="info",
            )
            save_predictions(traj_dir, instance_id, info)

        except KeyboardInterrupt:
            logger.info("Exiting InterCode environment...")
            env.close()
            break
        except Exception as e:
            traceback.print_exc()
            logger.warning(f"❌ Failed on {env.record['instance_id']}: {e}")
            env.reset_container()
            continue


In [9]:
agent = Agent("primary", args.agent)
env = SWEEnv(args.environment)


Generating test split: 100%|█████████████████████| 300/300 [00:00<00:00, 37608.08 examples/s]


INFO     💽 Loaded dataset from princeton-nlp/SWE-bench_Lite

INFO     Found image swe-agent with tags: ['swe-agent:latest'], created: 2024-04-02T23:37:55.180823208Z for linux  
         amd64.

DEBUG    Starting container with command: docker run -i --rm --name swe-agent-0ac67fae8d swe-agent /bin/bash -l -m

INFO     🌱 Environment Initialized

In [10]:
args.split

'dev'

In [38]:
env.data[0].keys()


dict_keys(['repo', 'instance_id', 'base_commit', 'patch', 'test_patch', 'problem_statement', 'hints_text', 'created_at', 'version', 'FAIL_TO_PASS', 'PASS_TO_PASS', 'environment_setup_commit'])

In [44]:
import pandas as pd
df = pd.DataFrame(env.data)

print (len(df))
df


23


,repo,instance_id,base_commit,patch,test_patch,problem_statement,hints_text,created_at,version,FAIL_TO_PASS,PASS_TO_PASS,environment_setup_commit
0,sqlfluff/sqlfluff,sqlfluff__sqlfluff-1625,14e1a23a3166b9a645a16de96f694c77a5d4abb7,diff --git a/src/sqlfluff/rules/L031.py b/src/...,diff --git a/test/cli/commands_test.py b/test/...,"TSQL - L031 incorrectly triggers ""Avoid using ...","Actually, re-reading the docs I think this is ...",2021-10-13T11:35:29Z,0.6,"[""test/cli/commands_test.py::test__cli__comman...","[""test/cli/commands_test.py::test__cli__comman...",67023b85c41d23d6c6d69812a41b207c4f8a9331
1,sqlfluff/sqlfluff,sqlfluff__sqlfluff-2419,f1dba0e1dd764ae72d67c3d5e1471cf14d3db030,diff --git a/src/sqlfluff/rules/L060.py b/src/...,diff --git a/test/rules/std_L060_test.py b/tes...,Rule L060 could give a specific error message\...,"@tunetheweb Yeah definitely, should be a prett...",2022-01-22T12:21:52Z,0.8,"[""test/rules/std_L060_test.py::test__rules__st...",[],a5c4eae4e3e419fe95460c9afd9cf39a35a470c4
2,sqlfluff/sqlfluff,sqlfluff__sqlfluff-1733,a1579a16b1d8913d9d7c7d12add374a290bcc78c,diff --git a/src/sqlfluff/rules/L039.py b/src/...,diff --git a/test/rules/std_L003_L036_L039_com...,Extra space when first field moved to new line...,Does running `sqlfluff fix` again correct the ...,2021-10-22T18:23:33Z,0.6,"[""test/rules/std_L003_L036_L039_combo_test.py:...","[""test/rules/std_L016_L36_combo_test.py::test_...",67023b85c41d23d6c6d69812a41b207c4f8a9331
3,sqlfluff/sqlfluff,sqlfluff__sqlfluff-1517,304a197829f98e7425a46d872ada73176137e5ae,diff --git a/src/sqlfluff/core/parser/helpers....,diff --git a/test/dialects/ansi_test.py b/test...,"""Dropped elements in sequence matching"" when d...",Sounds similar to #1458 where we should handle...,2021-10-06T07:57:35Z,0.6,"[""test/dialects/ansi_test.py::test__dialect__a...","[""test/dialects/ansi_test.py::test__dialect__a...",67023b85c41d23d6c6d69812a41b207c4f8a9331
4,sqlfluff/sqlfluff,sqlfluff__sqlfluff-1763,a10057635e5b2559293a676486f0b730981f037a,diff --git a/src/sqlfluff/core/linter/linted_f...,diff --git a/test/core/linter_test.py b/test/c...,dbt postgres fix command errors with UnicodeEn...,I get a dbt-related error -- can you provide y...,2021-10-26T17:28:28Z,0.6,"[""test/core/linter_test.py::test_safe_create_r...","[""test/core/linter_test.py::test__linter__path...",67023b85c41d23d6c6d69812a41b207c4f8a9331
5,marshmallow-code/marshmallow,marshmallow-code__marshmallow-1359,b40a0f4e33823e6d0f341f7e8684e359a99060d1,diff --git a/src/marshmallow/fields.py b/src/m...,diff --git a/tests/test_fields.py b/tests/test...,3.0: DateTime fields cannot be used as inner f...,Thanks for reporting. I don't think I'll have ...,2019-08-21T15:45:13Z,3.0,"[""tests/test_fields.py::TestParentAndName::tes...","[""tests/test_fields.py::test_field_aliases[Int...",8b3a32614fd4a74e93e9a63a042e74c1fea34466
6,marshmallow-code/marshmallow,marshmallow-code__marshmallow-1343,2be2d83a1a9a6d3d9b85804f3ab545cecc409bb0,diff --git a/src/marshmallow/schema.py b/src/m...,diff --git a/tests/test_marshalling.py b/tests...,[version 2.20.0] TypeError: 'NoneType' object ...,Thanks for reporting. I was able to reproduce ...,2019-08-13T04:36:01Z,2.20,"[""tests/test_marshalling.py::TestUnmarshaller:...","[""tests/test_marshalling.py::test_missing_is_f...",7015fc4333a2f32cd58c3465296e834acd4496ff
7,pvlib/pvlib-python,pvlib__pvlib-python-1707,40e9e978c170bdde4eeee1547729417665dbc34c,diff --git a/pvlib/iam.py b/pvlib/iam.py\n--- ...,diff --git a/pvlib/tests/test_iam.py b/pvlib/t...,regression: iam.physical returns nan for aoi >...,,2023-03-24T10:46:42Z,0.9,"[""pvlib/tests/test_iam.py::test_physical_n1_L0""]","[""pvlib/tests/test_iam.py::test_ashrae"", ""pvli...",6072e0982c3c0236f532ddfa48fbf461180d834e
8,pvlib/pvlib-python,pvlib__pvlib-python-1072,04a523fafbd61bc2e49420963b84ed8e2bd1b3cf,diff --git a/pvlib/temperature.py b/pvlib/temp...,diff --git a/pvlib/tests/test_temperature.py b...,temperature.fuentes errors when given tz-aware...,

In [ ]:
from run import should_skip

traj_dir = "trajectories/root/azure-gpt4__SWE-bench_Lite__default__t-0.00__p-0.95__c-2.00__install-1__run_dev_3/"
intance_id = env.data[0]["instance_id"]

should_skip(args, traj_dir=traj_dir, instance_id=intance_id)


In [42]:
display(Markdown(df['problem_statement'][0]))


TSQL - L031 incorrectly triggers "Avoid using aliases in join condition" when no join present
## Expected Behaviour

Both of these queries should pass, the only difference is the addition of a table alias 'a':

1/ no alias

```
SELECT [hello]
FROM
    mytable
```

2/ same query with alias

```
SELECT a.[hello]
FROM
    mytable AS a
```

## Observed Behaviour

1/ passes
2/ fails with: L031: Avoid using aliases in join condition.

But there is no join condition :-)

## Steps to Reproduce

Lint queries above

## Dialect

TSQL

## Version

sqlfluff 0.6.9
Python 3.6.9

## Configuration

N/A


In [11]:
env.get_available_actions()


[]

In [12]:
agent.get_environment_vars(env)


{'WINDOW': '',
 'OVERLAP': '',
 'CURRENT_LINE': '',
 'CURRENT_FILE': '',
 'SEARCH_RESULTS': '',
 'SEARCH_FILES': '',
 'SEARCH_INDEX': ''}

In [24]:
agent.command_patterns


{'open': re.compile(r'^\s*(open)\s*(.*?)$', re.MULTILINE|re.UNICODE),
 'goto': re.compile(r'^\s*(goto)\s*(.*?)$', re.MULTILINE|re.UNICODE),
 'scroll_down': re.compile(r'^\s*(scroll_down)\s*(.*?)$',
            re.MULTILINE|re.UNICODE),
 'scroll_up': re.compile(r'^\s*(scroll_up)\s*(.*?)$', re.MULTILINE|re.UNICODE),
 'create': re.compile(r'^\s*(create)\s*(.*?)$', re.MULTILINE|re.UNICODE),
 'submit': re.compile(r'^\s*(submit)\s*(.*?)^(None)\s*$',
            re.MULTILINE|re.DOTALL|re.UNICODE),
 'search_dir': re.compile(r'^\s*(search_dir)\s*(.*?)$',
            re.MULTILINE|re.UNICODE),
 'search_file': re.compile(r'^\s*(search_file)\s*(.*?)$',
            re.MULTILINE|re.UNICODE),
 'find_file': re.compile(r'^\s*(find_file)\s*(.*?)$', re.MULTILINE|re.UNICODE),
 'edit': re.compile(r'^\s*(edit)\s*(.*?)^(end_of_edit)\s*$',
            re.MULTILINE|re.DOTALL|re.UNICODE)}

In [30]:
agent_config = agent.config.to_dict()
agent_config.keys()


dict_keys(['system_template', 'instance_template', 'next_step_template', 'next_step_no_output_template', 'strategy_template', 'demonstration_template', 'demonstrations', 'put_demos_in_history', 'format_error_template', 'command_files', 'env_variables', 'util_functions', 'submit_command', 'parse_function', 'parse_command', 'history_processor', 'history_processor_args', 'command_docs', 'blocklist_error_template', 'blocklist', 'blocklist_standalone', 'state_command', '_commands', '_subroutines', 'subroutine_types'])

In [45]:
display(Markdown(agent_config['system_template']))


SETTING: You are an autonomous programmer, and you're working directly in the command line with a special interface.

The special interface consists of a file editor that shows you {WINDOW} lines of a file at a time.
In addition to typical bash commands, you can also use the following commands to help you navigate and edit files.

COMMANDS:
{command_docs}

Please note that THE EDIT COMMAND REQUIRES PROPER INDENTATION. 
If you'd like to add the line '        print(x)' you must fully write that out, with all those spaces before the code! Indentation is important and code that is not indented correctly will fail and require fixing before it can be run.

RESPONSE FORMAT:
Your shell prompt is formatted as follows:
(Open file: <path>) <cwd> $

You need to format your output using two fields; discussion and command.
Your output should always include _one_ discussion and _one_ command field EXACTLY as in the following example:
DISCUSSION
First I'll start by using ls to see what files are in the current directory. Then maybe we can look at some relevant files to see what they look like.
```
ls -a
```

You should only include a *SINGLE* command in the command section and then wait for a response from the shell before continuing with more discussion and commands. Everything you include in the DISCUSSION section will be saved for future reference.
If you'd like to issue two commands at once, PLEASE DO NOT DO THAT! Please instead first submit just the first command, and then after receiving a response you'll be able to issue the second command. 
You're free to use any other bash commands you want (e.g. find, grep, cat, ls, cd) in addition to the special commands listed above.
However, the environment does NOT support interactive session commands (e.g. python, vim), so please do not invoke them.

In [46]:
display(Markdown(agent_config['command_docs']))


open:
  docstring: opens the file at the given path in the editor. If line_number is provided, the window will be move to include that line
  signature: open <path> [<line_number>]
  arguments:
    - path (string) [required]: the path to the file to open
    - line_number (integer) [optional]: the line number to move the window to (if not provided, the window will start at the top of the file)

goto:
  docstring: moves the window to show <line_number>
  signature: goto <line_number>
  arguments:
    - line_number (integer) [required]: the line number to move the window to

scroll_down:
  docstring: moves the window down {WINDOW} lines
  signature: scroll_down

scroll_up:
  docstring: moves the window down {WINDOW} lines
  signature: scroll_down

create:
  docstring: creates and opens a new file with the given name
  signature: create <filename>
  arguments:
    - filename (string) [required]: the name of the file to create

submit:
  docstring: submits your current code and terminates the session
  signature: submit

search_dir:
  docstring: searches for search_term in all files in dir. If dir is not provided, searches in the current directory
  signature: search_dir <search_term> [<dir>]
  arguments:
    - search_term (string) [required]: the term to search for
    - dir (string) [optional]: the directory to search in (if not provided, searches in the current directory)

search_file:
  docstring: searches for search_term in file. If file is not provided, searches in the current open file
  signature: search_file <search_term> [<file>]
  arguments:
    - search_term (string) [required]: the term to search for
    - file (string) [optional]: the file to search in (if not provided, searches in the current open file)

find_file:
  docstring: finds all files with the given name in dir. If dir is not provided, searches in the current directory
  signature: find_file <file_name> [<dir>]
  arguments:
    - file_name (string) [required]: the name of the file to search for
    - dir (string) [optional]: the directory to search in (if not provided, searches in the current directory)

edit:
  docstring: replaces lines <start_line> through <end_line> (inclusive) with the given text in the open file. The replacement text is terminated by a line with only end_of_edit on it. All of the <replacement text> will be entered, so make sure your indentation is formatted properly. Python files will be checked for syntax errors after the edit. If the system detects a syntax error, the edit will not be executed. Simply try to edit the file again, but make sure to read the error message and modify the edit command you issue accordingly. Issuing the same command a second time will just lead to the same error message again.
  signature: edit <start_line>:<end_line>
<replacement_text>
end_of_edit
  arguments:
    - start_line (integer) [required]: the line number to start the edit at
    - end_line (integer) [required]: the line number to end the edit at (inclusive)
    - replacement_text (string) [required]: the text to replace the current selection with



In [47]:
display(Markdown(agent_config['instance_template']))


We're currently solving the following issue within our repository. Here's the issue text:
ISSUE:
{issue}

INSTRUCTIONS:
Now, you're going to solve this issue on your own. Your terminal session has started and you're in the repository's root directory. You can use any bash commands or the special interface to help you. Edit all the files you need to and run any checks or tests that you want. 
Remember, YOU CAN ONLY ENTER ONE COMMAND AT A TIME. You should always wait for feedback after every command. 
When you're satisfied with all of the changes you've made, you can submit your changes to the code base by simply running the submit command.
Note however that you cannot use any interactive session commands (e.g. python, vim) in this environment, but you can write scripts and run them. E.g. you can write a python script and then run it with `python <script_name>.py`.

NOTE ABOUT THE EDIT COMMAND: Indentation really matters! When editing a file, make sure to insert appropriate indentation before each line! 

IMPORTANT TIPS:
1. Always start by trying to replicate the bug that the issues discusses. 
   If the issue includes code for reproducing the bug, we recommend that you re-implement that in your environment, and run it to make sure you can reproduce the bug.
   Then start trying to fix it.
   When you think you've fixed the bug, re-run the bug reproduction script to make sure that the bug has indeed been fixed.
   
   If the bug reproduction script does not print anything when it succesfully runs, we recommend adding a print("Script completed successfully, no errors.") command at the end of the file,
   so that you can be sure that the script indeed ran fine all the way through. 

2. If you run a command and it doesn't work, try running a different command. A command that did not work once will not work the second time unless you modify it!

3. If you open a file and need to get to an area around a specific line that is not in the first 100 lines, say line 583, don't just use the scroll_down command multiple times. Instead, use the goto 583 command. It's much quicker. 
   
4. If the bug reproduction script requires inputting/reading a specific file, such as buggy-input.png, and you'd like to understand how to input that file, conduct a search in the existing repo code, to see whether someone else has already done that. Do this by running the command: find_file "buggy-input.png" If that doensn't work, use the linux 'find' command. 

5. Always make sure to look at the currently open file and the current working directory (which appears right after the currently open file). The currently open file might be in a different directory than the working directory! Note that some commands, such as 'create', open files, so they might change the current  open file.

6. When editing files, it is easy to accidentally specify a wrong line number or to write code with incorrect indentation. Always check the code after you issue an edit to make sure that it reflects what you wanted to accomplish. If it didn't, issue another command to fix it.
   

(Open file: {open_file})
(Current directory: {working_dir})
bash-$

In [48]:
agent_config['next_step_template']


'{observation}\n(Open file: {open_file})\n(Current directory: {working_dir})\nbash-$'

In [49]:
agent_config['next_step_no_output_template'] 


'Your command ran successfully and did not produce any output.\n(Open file: {open_file})\n(Current directory: {working_dir})\nbash-$'

In [50]:
agent_config['demonstration_template'] 


'Here is a demonstration of how to correctly accomplish this task.\nIt is included to show you how to correctly use the interface.\nYou do not need to follow exactly what is done in the demonstration.\n--- DEMONSTRATION ---\n{demonstration}\n--- END OF DEMONSTRATION ---\n'

In [51]:
agent_config['_subroutines'] 


{}